In [1]:
import sys
from pyspark.sql import functions as F
from pyspark.sql.functions import * 
from awsglue.job import Job
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from functools import reduce

sc = SparkContext.getOrCreate()
gc = GlueContext(sc)



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4,application_1643278496746_0005,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# source_df=spark.read.option("header","true").csv("s3://sol-dev-source/TruCape-Invoices")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:

source_dyf = gc.create_dynamic_frame.from_options(
    's3',
    {
        "paths": [
            's3://sol-dev-source/TruCape-Invoices'
        ],
        "recurse" : True
    },
    "csv",
    transformation_ctx = "source_dyf",
    headerText = True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
source_df = source_dyf.toDF()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# (b) Create array with desired columns
old_columns = source_df.schema.names
new_columns = [
    field.lower().replace("col0", "External Reference")
    .replace("col1", "Number")
    .replace("col2", "Counterparty")
    .replace("col3", "Amount Due")
    .replace("col4", "Currency")
    .replace("col5", "Cost Currency")
    .replace("col6", "Costing Rate")
    .replace("col7", "Due Date")
    .replace("col8", "Issue Date")
    for field in old_columns
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# (c) Overwrite and persist `new_columns`
source_df = reduce(
    lambda source_df, idx: source_df.withColumnRenamed(old_columns[idx], new_columns[idx]),
    range(len(old_columns)),
    source_df,
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
source_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+--------------------+------------+--------+-------------+------------+--------+----------+
|External Reference|Number|        Counterparty|  Amount Due|Currency|Cost Currency|Costing Rate|Due Date|Issue Date|
+------------------+------+--------------------+------------+--------+-------------+------------+--------+----------+
|External Reference|Number|        Counterparty| Amount Due |Currency|Cost Currency|Costing Rate|Due Date|Issue Date|
|           SC23677|131527|Russian Venture I...|  17,401.50 |     USD|          ZAR|     15.2174|  17-Oct|    23-Aug|
|           SC23678|131731|Russian Venture I...|  17,617.50 |     USD|          ZAR|     15.2174|  17-Oct|    23-Aug|
|           SC23698|131722|        JWM Asia Ltd|  16,464.00 |     USD|          ZAR|     15.2174|  22-Oct|    23-Aug|
|           SC23761|131984|        JWM Asia Ltd|  12,936.00 |     USD|          ZAR|     15.2174|  22-Oct|    23-Aug|
|           SC23853|132113|        JWM Asia Ltd|  16,464

In [7]:
pandas_source_df = source_df.toPandas()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
print(source_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[external reference: string, number: string, counterparty: string, amount due: string, currency: string, cost currency: string, costing rate: string, due date: string, Issue Date: string]

In [8]:
source_df = pandas_source_df.iloc[1: , :]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
source_df=spark.createDataFrame(source_df) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
source_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+--------------------+-----------+--------+-------------+------------+--------+----------+
|External Reference|Number|        Counterparty| Amount Due|Currency|Cost Currency|Costing Rate|Due Date|Issue Date|
+------------------+------+--------------------+-----------+--------+-------------+------------+--------+----------+
|           SC23677|131527|Russian Venture I...| 17,401.50 |     USD|          ZAR|     15.2174|  17-Oct|    23-Aug|
|           SC23678|131731|Russian Venture I...| 17,617.50 |     USD|          ZAR|     15.2174|  17-Oct|    23-Aug|
|           SC23698|131722|        JWM Asia Ltd| 16,464.00 |     USD|          ZAR|     15.2174|  22-Oct|    23-Aug|
|           SC23761|131984|        JWM Asia Ltd| 12,936.00 |     USD|          ZAR|     15.2174|  22-Oct|    23-Aug|
|           SC23853|132113|        JWM Asia Ltd| 16,464.00 |     USD|          ZAR|     15.2174|  22-Oct|    23-Aug|
|           SC23850|132135|        JWM Asia Ltd| 15,523.20 |    

In [11]:
# ADD YEAR TO DATE COLUMNS
due_date="due date"
issue_date="issue date"
year = F.lit("-2022")
due_date_add_year = F.concat(col(due_date),year)
issue_date_add_year = F.concat(col(issue_date),year)
                               
source_df = source_df.withColumn(due_date, due_date_add_year) \
    .withColumn(issue_date, issue_date_add_year)
    


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# SET DECIMAL PLACES
costing_rate="costing rate"
format_costing_rate = format_number(round(costing_rate ,4), 4)
amount_due="amount due"
format_amount_due=format_number(round(amount_due ,2), 2)

source_df = source_df.withColumn(costing_rate, format_costing_rate) \
# amount due gives null value - not sure why

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# FORMAT DATES
source_df = source_df.select(
    col("external reference"), \
    col("number"), \
    col("counterparty"), \
    col("amount due"), \
    col("currency"), \
    col("cost currency"), \
    col("costing rate"), \
    to_date(F.col("due date"), "d-MMM-yyy").alias("due date"), \
    to_date(F.col("issue date"), "d-MMM-yyy").alias("issue date")) \
    


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
source_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+--------------------+-----------+--------+-------------+------------+----------+----------+
|external reference|number|        counterparty| amount due|currency|cost currency|costing rate|  due date|issue date|
+------------------+------+--------------------+-----------+--------+-------------+------------+----------+----------+
|           SC23677|131527|Russian Venture I...| 17,401.50 |     USD|          ZAR|     15.2174|2022-10-17|2022-08-23|
|           SC23678|131731|Russian Venture I...| 17,617.50 |     USD|          ZAR|     15.2174|2022-10-17|2022-08-23|
|           SC23698|131722|        JWM Asia Ltd| 16,464.00 |     USD|          ZAR|     15.2174|2022-10-22|2022-08-23|
|           SC23761|131984|        JWM Asia Ltd| 12,936.00 |     USD|          ZAR|     15.2174|2022-10-22|2022-08-23|
|           SC23853|132113|        JWM Asia Ltd| 16,464.00 |     USD|          ZAR|     15.2174|2022-10-22|2022-08-23|
|           SC23850|132135|        JWM Asia Ltd|

In [16]:
# source_df.write.csv(path='s3://sol-dev-output/TruCape-Invoices', mode='overwrite')
source_df.coalesce(1).write.csv(path='s3://sol-dev-output/TruCape-Invoices', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…